<h1><center>Кредитный отдел банка</center></h1>

## Введение
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — ответить на четыре вопроса:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о поведении пользователей получим из файла `data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

 
Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Ответы на вопросы



Описание проекта:
Отток клиентов
Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.
Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.

## Обзор данных

Откроем данные и посмотрим общую информацию.

In [5]:
import pandas as pd
import urllib.request

from pymystem3 import Mystem
from collections import Counter

from os import path
from pathlib import Path


In [6]:
YANDEX_DATASETS_PATH = 'https://code.s3.yandex.net/datasets/'
dataset_folder = 'datasets'
dataset_name = 'data.csv'

#download dataset if not existed
if not path.exists(dataset_folder + '/' + dataset_name):
    #create dir if not existed
    Path(dataset_folder).mkdir(parents=True, exist_ok=True)

    #download dataset
    urllib.request.urlretrieve(YANDEX_DATASETS_PATH + dataset_name,
                               dataset_folder + '/' + dataset_name)

df = pd.read_csv(dataset_folder + '/' + dataset_name)
df.info()

# Read file
data = pd.read_csv('datasets/data.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------    

In [7]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Согласно документации к данным:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита
    
 

Вывод:
В двух колонках days_employed и total_income присутствуют пропуски. Debt(имел ли задолженность по возврату кредитов) - это основная колонка, значение которой мы будем использовать для каждого из наших исследований.
1. Для первой гипотезы сравним значения в children и debt
2. Для второй гипотезы сравним значения в family_status и debt
3. Для третьей гипотезы сравним значения в total_income и debt
4. Для четвертой гипотезы сравним значения в purpose и debt

Чтобы это правильно сделать потребуется предварительная предобработка данных.

## Предобработка данных

### Стиль заголовков

Посмотрим на стиль заголовков, исключим пропуски.

In [8]:
print(data.columns)

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


Вывод: пропусков нет, каждая колонка имеет тип snake_case.

### Обработка пропусков

Посчитаем количество пропусков.

In [9]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропусков в двух колонок одинаково. Вероятно, что это по два пропуска в каждой из этих строчек. Проверим это.

In [10]:
data_nan = data[data['days_employed'].isna()]
data_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


In [11]:
data_nan.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Пропуску в колонке days_employed соответствует пропуск в колонке total_income. количество пропусков 2174 - это примерно 10% нашей выборки. Скорее всего, пропуски и отрицательные значения в days_employed вызваны неправильной работой с датами. В формуле расчета total_income по-видимому участвует days_employed, и соответственно эти значения тоже принимают значение NaN. Значения total_income нужны для ответа на 3-ий вопрос. Их удаление может исказить выводы наших расчетов.
Total_income - количественная переменная. Пропуски в таких переменных заполняют характерными значениями. В нашем случае заменим пропущенные значения медианными для каждого из типа занятости.

Посмотрим, какие типы занятости входят в пропущенные значения.

In [12]:
data_nan['income_type'].unique()

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

Составим на основании этого список.

In [13]:
income_type_list = ['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель']

Напишем функцию, которая заменяет пропущенные значения медианными для каждого типа занятости из списка.

In [14]:
#function that change missing values to median one for every income_type
def median_income_type(income_type_list):
    for income_name in income_type_list:

        #count median for each type
        median_income = data[data['income_type'] == income_name]['total_income'].median()

        #c - condition
        с = (data['income_type'] == income_name) & (data['total_income'].isnull())

        #fill NaN with median value
        data.loc[с, 'total_income'] = data.loc[с, 'total_income'].fillna(median_income)

Передадим функции наш список

In [15]:
median_income_type(income_type_list)

Заменим пропущенные значения в days_employed нулями, так как эти данные не нужны для нашего исследования.

In [16]:
data['days_employed'] = data['days_employed'].fillna(0)

Посмотрим на результат:

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Посмотрим на уникальные значения в колонках с категореальными значениями.

In [18]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Имеются дубликаты, связанные с регистром. Пропусков нету.

In [19]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Пропусков нету.

In [20]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Имеется пропущенное значение. Скорее всего человеческий фактор. По нашей таблице невозможно его восстановить. Заменим его на любое из gender.

In [21]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'

Проверим замену.

In [22]:
data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

In [23]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Пропусков нету.

In [24]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значение '20' скорее всего вызвано тем, что приписали лишний '0'. На самом деле это '2'. А '-1' это единица. Скорее всего, обе ошибки вызваны человеческим фактором. Исправим.

In [25]:
#data.loc[data['children'] == 20, 'children'] = 2
#data.loc[data['children'] == -1, 'children'] = 1
data['children'] = data['children'].replace([20, -1], [2, 1])

Проверим замены.

In [26]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [27]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Есть множество значений для лемматизации и категоризации. Пропусков нету.

In [28]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Пропусков нету.

Вывод: Пропуски в категориальных переменных скорее всего вызваны человеческим фактором. Пропуски в количественных переменных вызваны или ошибкой в формулах расчета, или, например, ошибкой импортирования данных из другой базы.

### Замена типа данных

Рассмотрим типы переменных.

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


days_employed, total_income в вещественном формате. Количество дней должно быть целочисленным, но так как в нашем исследовании эти данные не используются, не будем менять их тип. Доход переведем в целочисленный формат для простоты расчета. Для замены из вещественного формата в целочисленный в pandas используется метод astype()

In [30]:
data['total_income'] = data['total_income'].astype('int')

Проверим замену типа переменной.

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


In [32]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Проверим, есть ли отрицательные, артефактные значения в total_income.

In [33]:
data['total_income'].min()

20667

Вывод: Данные в total_income приведены в целочисленный формат и не имеют отрицательных значений.

### Обработка дубликатов

Переведем значения в education в нижний регистр

In [34]:
data['education'] = data['education'].str.lower()

Проверим замену.

In [35]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим, совпадают ли эти значения с education_id.

In [36]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Посмотрим, имеются ли дупликаты в записях.

In [37]:
data.duplicated().sum()

71

Скорее всего, они появились там, где мы заменяли пропуски в days_employed и total_income. Потому что, в такой большой выборке присутствует высокий шанс, что найдутся значения с одинаковым набором категориальных параметров в отсутствии разброса в числовых.

Вывод: Дупликаты в значениях purpose вызваны человеским фактором. Дупликаты в нашей базе получились искусственным образом при работе с пропусками. Их удаление может повлиять на результаты исследований. Не будем этого делать.

### Лемматизация

Посмотрим на уникальные значения в 'purpose'

In [38]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Много общих по смыслу значений. Посмотрим, какие леммы встречаются наиболее часто. А также сделаем словарь который связывает уникальное значение в purpose с его лемматизацией. Это понадобится нам для дальнейшей категоризации.

In [39]:
lemmas = []
lemmas_dict = {}
m = Mystem()

for unique in data['purpose'].unique():
    unique_lemmas = m.lemmatize(unique)
    lemmas_dict[unique] = unique_lemmas
    lemmas.extend(unique_lemmas)

print(Counter(lemmas))

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


Вывод: Частотные леммы, которые можно выделить в отдельные группы: "недвижимость"(сюда также отнесем "жилье"), "образование", "автомобиль", "свадьба". Они покрывают всю выборку значений в purpose.

### Категоризация данных

Для значений из purpose выделим 4 большие категории: 'недвижимость', 'свадьба', 'автомобиль', 'образование'. Для этого напишем функцию, которая, используя полученный словарь с лемматизацией, для каждого значения из purpose, присвоет нужную категорию.

In [40]:
def group_purpose(row):
    for value in lemmas_dict[row['purpose']]:
        if value == 'свадьба':
            return 'свадьба'
        elif ( (value == 'недвижимость') or (value == 'жилье') ):
            return 'недвижимость'   
        elif value == 'автомобиль':
            return 'автомобиль'    
        elif value == 'образование':
            return 'образование'     

        

<div class="alert alert-info">Так и хотел вначале но, посмотрев на скорость лемматизации, отбросил эту идею) Не соглашусь с тем, что это сокращает время. Если мы лемматизировали только уникальные значения и сделали словарь, это должно быть точно быстрее, чем лемматизировать весь массив в 20к строчек. Мне кажется, даже с алгоритмической точки зрения, это правильный подход - зачем нам лемматизировать множество повторяющихся значений?</div>

In [41]:
import time

In [42]:
data['purpose_group'] = data.apply(group_purpose, axis=1) 

Проверим результат.

In [43]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


Сделаем категоризацию для total_income(доход). Для этого сгруппируем наши данные по типу дохода и рассмотрим общие значение для этой переменной.

In [44]:
data.groupby('income_type')['total_income'].mean()


income_type
безработный        131339.000000
в декрете           53829.000000
госслужащий        168837.299520
компаньон          199413.927237
пенсионер          135133.413641
предприниматель    499163.000000
сотрудник          159512.838205
студент             98201.000000
Name: total_income, dtype: float64

In [45]:
data.groupby('income_type')['total_income'].median()

income_type
безработный        131339.0
в декрете           53829.0
госслужащий        150447.0
компаньон          172357.0
пенсионер          118514.0
предприниматель    499163.0
сотрудник          142594.0
студент             98201.0
Name: total_income, dtype: float64

In [46]:
data.groupby('income_type')['total_income'].max()

income_type
безработный         202722
в декрете            53829
госслужащий         910451
компаньон          2265604
пенсионер           735103
предприниматель     499163
сотрудник          1726276
студент              98201
Name: total_income, dtype: int32

In [47]:
data.groupby('income_type')['total_income'].min()

income_type
безработный         59956
в декрете           53829
госслужащий         29200
компаньон           28702
пенсионер           20667
предприниматель    499163
сотрудник           21367
студент             98201
Name: total_income, dtype: int32

In [48]:
data.groupby('income_type')['total_income'].agg(['mean', 'median', 'max', 'min'])

,mean,median,max,min
income_type,,,,
безработный,131339.000000,131339.0,202722,59956
в декрете,53829.000000,53829.0,53829,53829
госслужащий,168837.299520,150447.0,910451,29200
компаньон,199413.927237,172357.0,2265604,28702
пенсионер,135133.413641,118514.0,735103,20667
предприниматель,499163.000000,499163.0,499163,499163
сотрудник,159512.838205,142594.0,1726276,21367
студент,98201.000000,98201.0,98201,98201


Применим метов cut() для расчета диапазонов категоризации.

In [49]:
pd.qcut(data['total_income'], 4)


0        (195549.0, 2265604.0]
1         (107798.0, 142594.0]
2         (142594.0, 195549.0]
3        (195549.0, 2265604.0]
4         (142594.0, 195549.0]
                 ...          
21520    (195549.0, 2265604.0]
21521     (142594.0, 195549.0]
21522    (20666.999, 107798.0]
21523    (195549.0, 2265604.0]
21524    (20666.999, 107798.0]
Name: total_income, Length: 21525, dtype: category
Categories (4, interval[float64, right]): [(20666.999, 107798.0] < (107798.0, 142594.0] < (142594.0, 195549.0] < (195549.0, 2265604.0]]

In [50]:
def group_total_income(row):
        total_income = row['total_income']
        if  0 <= total_income <= 107000:
            return 'низкий'
        elif 107000 < total_income <= 142000:
            return 'средний'   
        elif 142000 < total_income < 195000:
            return 'высокий'
        elif 195000 < total_income:
            return 'сверхвысокий'
          

Применим полученную функцию

In [51]:
data['total_income_group'] = data.apply(group_total_income, axis=1)

Проверим работу функции

In [52]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,total_income_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,сверхвысокий
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,высокий
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,сверхвысокий
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий


Вывод: Для числовых переменных рассмотрели общие значения, и на основе этого составили диапазоны для категорий. Для категориальных переменных воспользовались лемматизацией.

## Ответы на вопросы

Для ответа на поставленные вопросы сделаем сводную табличку для каждой рассматриваемой величины и наличием\отсутствием задолжностью по кредиту в прошлом. Добавим в каждую из таблиц столбец 'ratio' - отношение количества людей, имеющих опыт задолжности, к тем у кого ее не было. И отсортируем каждую из таблиц по этому значению.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [53]:
data_pivot_children = data.pivot_table(index=['children'], columns='debt', values='purpose', aggfunc='count', fill_value=0)
try:
    data_pivot_children['ratio'] = data_pivot_children[1] / (data_pivot_children[0] + data_pivot_children[1])
except:
    print('probably divided by zero')
display(data_pivot_children.sort_values('ratio'))

debt,0,1,ratio
children,,,
5,9,0,0.000000
0,13086,1063,0.075129
3,303,27,0.081818
1,4420,445,0.091470
2,1929,202,0.094791
4,37,4,0.097561


Вывод: Основная часть выборки приходится на людей не имеющих детей, имеющих одного ребенка, имеющих двух детей. Первая группа имеет более высокие шансы возврата, чем остальные две. При этом количество детей не имеет значения.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [54]:
data_pivot_family = data.pivot_table(index=['family_status'], columns='debt', values='purpose', aggfunc='count')
try:
    data_pivot_family['ratio'] = data_pivot_family[1] / (data_pivot_family[0] + data_pivot_family[1])
except:
    print('probably divided by zero')
display(data_pivot_family.sort_values('ratio'))

debt,0,1,ratio
family_status,,,
вдовец / вдова,897,63,0.065625
в разводе,1110,85,0.071130
женат / замужем,11449,931,0.075202
гражданский брак,3789,388,0.092890
Не женат / не замужем,2539,274,0.097405


Вывод: Люди, имеющие опыт брака \ находящиеся в браке, имеют более высокий шанс возврата кредита. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [55]:
data_pivot_income = data.pivot_table(index=['total_income_group'], columns='debt', values='purpose', aggfunc='count')
try:
    data_pivot_income['ratio'] = data_pivot_income[1] / (data_pivot_income[0] + data_pivot_income[1])
except:
    print('probably divided by zero')
display(data_pivot_income.sort_values('ratio'))

debt,0,1,ratio
total_income_group,,,
сверхвысокий,5035,389,0.071718
низкий,4884,423,0.079706
средний,4040,379,0.085766
высокий,5825,550,0.086275


Вывод: По мере роста уровня дохода, повышается шанс возврата кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [56]:
data_pivot_purpose = data.pivot_table(index=['purpose_group'], columns='debt', values='purpose', aggfunc='count')
try:
    data_pivot_purpose['ratio'] = data_pivot_purpose[1] / (data_pivot_purpose[0] + data_pivot_purpose[1])
except:
    print('probably divided by zero')
display(data_pivot_purpose.sort_values('ratio'))

debt,0,1,ratio
purpose_group,,,
недвижимость,10058,782,0.072140
свадьба,2162,186,0.079216
образование,3652,370,0.091994
автомобиль,3912,403,0.093395


In [57]:
data_pivot_purpose.style.format({'ratio': '{:.2%}'})

debt,0,1,ratio
purpose_group,,,
автомобиль,3912,403,9.34%
недвижимость,10058,782,7.21%
образование,3652,370,9.20%
свадьба,2162,186,7.92%


Вывод: Люди, берущие кредит на ремонт\покупку недвижимости имеют самый высокий шанс возврата кредита. Люди, организующие свадьбу, имеют более низкие шансы. Примерно равные, еще более низкие шансы у людей берущих кредит на получения образования или покупку автомобиля.

## Общий вывод

После обработки данных, выянилось, что часть данных в столбцах days_employed и total_income пропущенна. Пропуски, скорее всего вызваны ошибкой в работе с датами. Есть локальные пропуски в children и gender. Вероятно, просто опечатки. Имеются дупликаты, связанные с регистром в education.

Мы проверили четыре исследования и установили:
1. Наличие детей уменьшает шанс возврата кредита в срок. Это связано с тем, что бездетные клиенты не имеют той финансовой нагрузки, что клиенты с детьми.
2. Наличие брака увеличивает шансы клиентов вернуть кредит в срок. Это связано с тем, что при расторжения брака, на них остается часть кредитной нагрузки, в отличие от людей, законодательно не оформивших свои отношения.
3. Клиенты с более высоким уровнем дохода имеют более высокие шансы возврата кредита в срок.
4. Кленты, берущие кредит на недвижимость более ответственно относятся к его погашению, чем в остальных случаях. Для многих это риск потери жилой площади. Кредит на свадьбу подразумевает, что выплачивать его будет не один человек, а семья. Видимо поэтому, шансы его погашения выше, чем в случае кредита на образование или автомобиль.

Рекомендации:
- разобраться с причиной появления пропусков в days_employed и total_income.
- для колонок purpose и education запретить ручной ввод значений, вместо этого сделать выпадающий список по категориям.